<a href="https://colab.research.google.com/github/mahtabkarami/Projects/blob/main/Flask_braintumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# نصب کتابخانه‌ها
!pip install flask pyngrok kagglehub

# ایمپورت‌ها
import kagglehub
from flask import Flask, request, render_template_string, send_from_directory
from pyngrok import ngrok, conf
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, losses
import numpy as np
import os
import shutil
import threading
import time

# دانلود دیتاست
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Path to dataset files:", path)

# مسیر دیتا
train_data_dir = os.path.join(path, 'Training')
test_data_dir = os.path.join(path, 'Testing')

# دیتاجنریتور
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(64, 64),
    color_mode='grayscale', class_mode='categorical',
    batch_size=128, shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir, target_size=(64, 64),
    color_mode='grayscale', class_mode='categorical',
    batch_size=128, shuffle=False
)

# تعداد کلاس‌ها
num_classes = train_generator.num_classes

# ساخت مدل
myInput = layers.Input(shape=(64, 64, 1))
conv1 = layers.Conv2D(32, 3, padding='same', strides=2)(myInput)
bn1 = layers.BatchNormalization()(conv1)
ac1 = layers.Activation('relu')(bn1)
conv2 = layers.Conv2D(64, 3, padding='same', strides=2)(ac1)
bn2 = layers.BatchNormalization()(conv2)
ac2 = layers.Activation('relu')(bn2)
flat = layers.Flatten()(ac2)
out_layer = layers.Dense(num_classes, activation='softmax')(flat)

myModel = models.Model(myInput, out_layer)
myModel.compile(optimizer=optimizers.SGD(learning_rate=0.001),
                loss=losses.categorical_crossentropy,
                metrics=['accuracy'])

# آموزش مدل
history = myModel.fit(train_generator, epochs=50, validation_data=test_generator)

# ذخیره مدل
model_path = "/content/my_mri_model.h5"
myModel.save(model_path)
model = load_model(model_path)

# لیبل‌های کلاس
class_labels = list(train_generator.class_indices.keys())

# پوشه آپلود
upload_folder = '/content/uploads'
os.makedirs(upload_folder, exist_ok=True)

# ساخت اپلیکیشن Flask
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = upload_folder

html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Brain Tumor MRI Prediction</title>
</head>
<body style="text-align:center; font-family: Arial;">
    <h1>Upload an MRI Image</h1>
    <form method="POST" action="/" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*" required>
        <br><br>
        <input type="submit" value="Predict">
    </form>

    {% if image_url %}
        <h2>Uploaded Image:</h2>
        <img src="{{ image_url }}" style="width:300px; height:300px; object-fit:cover; border:2px solid black;">
        <h2>Predicted Tumor Type: {{ prediction }}</h2>
        <h3>Confidence: {{ confidence }}</h3>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def upload_predict():
    prediction = None
    confidence = None
    image_url = None
    if request.method == "POST":
        file = request.files["file"]
        if file:
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(filepath)

            img = image.load_img(filepath, target_size=(64, 64), color_mode='grayscale')
            img_array = image.img_to_array(img)
            img_array = img_array / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            pred = model.predict(img_array)
            pred_class = np.argmax(pred)
            pred_confidence = np.max(pred)

            prediction = class_labels[pred_class]
            confidence = f"{pred_confidence * 100:.2f}%"
            image_url = f"/uploads/{file.filename}"

    return render_template_string(html_template, prediction=prediction, confidence=confidence, image_url=image_url)

@app.route("/uploads/<filename>")
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# تابع اجرای Flask
def run_flask():
    app.run(port=5000, host="0.0.0.0", debug=False, use_reloader=False)

# تابع اتصال و مانیتورینگ ngrok
def start_ngrok():
    os.system("ngrok authtoken 2wIkinplPcxwWDOumn7c2cFJYts_3fwGHsZW4gAByMoAUixQQ")
    while True:
        try:
            tunnel = ngrok.connect(5000)
            public_url = tunnel.public_url
            print("\nPublic URL:", public_url)
            print("آدرس بالا را باز کنید!")

            # ذخیره آدرس در فایل
            with open("/content/ngrok_url.txt", "w") as f:
                f.write(public_url)

            while True:
                time.sleep(10)  # بررسی سلامت هر ۱۰ ثانیه
        except Exception as e:
            print(f"\n\n[Warning] ngrok error: {e}")
            print("[Info] Restarting ngrok tunnel...")
            time.sleep(2)

# اجرای همه چیز
if __name__ == "__main__":
    threading.Thread(target=run_flask).start()
    threading.Thread(target=start_ngrok).start()

Path to dataset files: /kaggle/input/brain-tumor-mri-dataset
Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 25s 525ms/step - accuracy: 0.4964 - loss: 1.1625 - val_accuracy: 0.2647 - val_loss: 1.3606
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 26s 564ms/step - accuracy: 0.6929 - loss: 0.7944 - val_accuracy: 0.2288 - val_loss: 1.3841
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 21s 458ms/step - accuracy: 0.7573 - loss: 0.6583 - val_accuracy: 0.2288 - val_loss: 1.4332
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 22s 493ms/step - accuracy: 0.7684 - loss: 0.6114 - val_accuracy: 0.2288 - val_loss: 1.4992
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 22s 481ms/step - accuracy: 0.7950 - loss: 0.5598 - val_accuracy: 0.2288 - val_loss: 1.5474
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 21s 461ms/step - accuracy: 0.8205 - loss: 0.5216 - val_accuracy: 0.2372 - val_loss: 1.5267
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 42s 481ms/step - accuracy: 0.8164 - loss: 0

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [4]:

with open("/content/ngrok_url.txt", "r") as f:
    url = f.read()
print("Current Public URL:", url)

Current Public URL: https://0f9c-34-31-51-109.ngrok-free.app
